In [0]:
%pip install osmnx numpy<2.0.0
%restart_python

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.0/18.0 MB 72.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 83.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 99.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.7/27.7 MB 34.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.6/9.6 MB 69.7 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 2.1.3
    Not uninstalling numpy at /databricks/python3/lib/python3.12/site-packages, outside environment /local_disk0/.ephemeral_nfs/envs/pythonEnv-cb9e6239-f4fa-4527-bb35-3df6bbf9c3b4
    Can't uninstall 'numpy'. No files were found to uninstall.
Note: you may need to restart the kernel using %restart_python or dbutils.library.restartPython() to use updated packages.


In [0]:
import osmnx as ox
from shapely.geometry import LineString, Point

# ox.settings.use_cache = True
# ox.settings.cache_folder = "/Volumes/sandbox/danny_schema/dw_volume/osmnx_cache"

def get_route_with_wkt(start_latlng, end_latlng, route_name="Route"):
    """Get route between two points and return WKT data"""
    
    # Download the street network for the area
    G = ox.graph_from_point(start_latlng, dist=20000, network_type="drive") #Limit to 20KM
    
    # Get the nearest network nodes to the start and end points
    orig_node = ox.nearest_nodes(G, start_latlng[1], start_latlng[0])
    dest_node = ox.nearest_nodes(G, end_latlng[1], end_latlng[0])
    
    # Find the shortest path
    route = ox.shortest_path(G, orig_node, dest_node, weight="length")
    
    # Convert to WKT formats
    start_wkt = Point(start_latlng[1], start_latlng[0]).wkt  # longitude, latitude
    end_wkt = Point(end_latlng[1], end_latlng[0]).wkt
    
    # Convert route nodes to LineString WKT
    coords_list = [(G.nodes[node]['x'], G.nodes[node]['y']) for node in route]
    route_linestring = LineString(coords_list)
    route_wkt = route_linestring.wkt
    
    return {
        'route_name': route_name,
        'start_wkt': start_wkt,
        'end_wkt': end_wkt,
        'route_linestring_wkt': route_wkt
    }

# Example usage
start_latlng = (-38.6650072,143.0947316)  # Twelve Apostles
end_latlng = (-38.6194185,142.9107861)    # The Grotto

# Get route data
route_data = get_route_with_wkt(start_latlng, end_latlng, "Great Ocean Road")

# Create DataFrame from list of dictionaries
routes_data = [route_data]  # You can add multiple routes here
df = spark.createDataFrame(routes_data)

display(df)
     

end_wkt,route_linestring_wkt,route_name,start_wkt
POINT (142.9107861 -38.6194185),"LINESTRING (143.1031691 -38.6617046, 143.059574 -38.6390937, 143.0481514 -38.6342619, 143.0347077 -38.6275568, 143.0009273 -38.6209476, 142.9999391 -38.6196515, 142.9990803 -38.6187409, 142.9982478 -38.617906, 142.9988766 -38.6173765, 142.9988942 -38.617342, 143.0003471 -38.6121976, 142.9977344 -38.6121118, 142.9749833 -38.6179684, 142.9521373 -38.617077, 142.9104222 -38.6126272)",Great Ocean Road,POINT (143.0947316 -38.6650072)


In [0]:
%sql
CREATE OR REPLACE TABLE dw_vicroads_demo.default.sample_points (
    id BIGINT,
    name STRING, 
    location STRING,
    created_date TIMESTAMP
)
--TBLPROPERTIES ('delta.minReaderVersion' = '3', 'delta.minWriterVersion' = '7');

In [0]:
%sql
INSERT INTO dw_vicroads_demo.default.sample_points VALUES
    (1, 'Melbourne CBD', 'POINT(144.9631 -37.8136)', current_timestamp()),
    (2, 'Sydney Opera House', 'POINT(151.2153 -33.8568)', current_timestamp()),
    (3, 'Brisbane', 'POINT(153.0251 -27.4698)', current_timestamp()),
    (4, 'Adelaide', 'POINT(138.6007 -34.9285)', current_timestamp()),
    (5, 'Perth', 'POINT(115.8575 -31.9505)', current_timestamp()),
    (6, 'Hobart', 'POINT(147.3272 -42.8821)', current_timestamp()),
    (7, 'Darwin', 'POINT(130.8456 -12.4634)', current_timestamp()),
    (8, 'Canberra', 'POINT(149.1300 -35.2809)', current_timestamp()),
    (9, 'Great Ocean Road', 'LINESTRING (143.1031691 -38.6617046, 143.059574 -38.6390937, 143.0481514 -38.6342619, 143.0347077 -38.6275568, 143.0009273 -38.6209476, 142.9999391 -38.6196515, 142.9990803 -38.6187409, 142.9982478 -38.617906, 142.9988766 -38.6173765, 142.9988942 -38.617342, 143.0003471 -38.6121976, 142.9977344 -38.6121118, 142.9749833 -38.6179684, 142.9521373 -38.617077, 142.9104222 -38.6126272)', current_timestamp()),
    (11, 'Box Hill South', 'POLYGON((145.1101764090017 -37.84306990099927,145.11111641100106 -37.837662656995484,145.11138415400546 -37.837517232996305,145.11216842999673 -37.83317850199773,145.11259073399648 -37.83004788600227,145.11289142199794 -37.82770585099739,145.11291181400028 -37.82770797599658,145.11866026599634 -37.82837451699849,145.121835836999 -37.82872743600293,145.13112053000455 -37.82978200799988,145.13294919499592 -37.829994496998076,145.1330776899947 -37.83002774599787,145.1363012119996 -37.830388628998826,145.13630216900185 -37.83038864800434,145.13630019899944 -37.83039556700381,145.13629420800197 -37.83042926699886,145.13592563799563 -37.83267700299586,145.13583979100244 -37.83316686999659,145.1356715520024 -37.83408468900002,145.13549786300024 -37.835013610997215,145.13510690199558 -37.83711304700421,145.13474395000415 -37.839018060000924,145.13385097099513 -37.84385102699721,145.13349212000034 -37.84581504999854,145.13073951599583 -37.84550329700258,145.1253504260014 -37.84488889200363,145.1210430080035 -37.844381482996546,145.12110767499644 -37.84401003600047,145.12124858599617 -37.8431870489987,145.12128273499877 -37.84306134499982,145.12162662299463 -37.84124081300439,145.1214982600032 -37.84122913900386,145.12136999500157 -37.84124415999653,145.12126738499546 -37.84127786200289,145.12089355500382 -37.84131531300416,145.12067539400488 -37.841392664001404,145.12061857399686 -37.84141844099759,145.12047094999994 -37.8414157819973,145.12033787899472 -37.841349320997786,145.12028202599632 -37.84130364299894,145.1201679370056 -37.84126692000153,145.11993903200474 -37.84126637800369,145.11982231699756 -37.841337539996026,145.11979868699726 -37.841409092002316,145.11966102100237 -37.84147853800004,145.11961433100038 -37.84153105300416,145.11953464599685 -37.84157354200062,145.11948746399568 -37.841689332999195,145.11940701899738 -37.841786031996826,145.11933721899936 -37.84183006999715,145.11917588900172 -37.84196324799969,145.11909409100335 -37.84206013000256,145.11900029900144 -37.84219401200342,145.11876316800212 -37.8423783600005,145.11873442700164 -37.842480164997504,145.118794782003 -37.842543664002505,145.11881529699687 -37.842568224000814,145.11883257200452 -37.84259409399551,145.11883689700068 -37.8426044550031,145.11883820300238 -37.84261567799791,145.11883751599814 -37.84262339899942,145.11883480300511 -37.84263363799648,145.1188283070023 -37.842646664999144,145.1188232539954 -37.84265346000055,145.11880994800197 -37.84266784699954,145.11879528299752 -37.842680761000096,145.11878458500078 -37.84268846700372,145.1187689540016 -37.842697553002125,145.1186575900033 -37.842750754997844,145.1186208020036 -37.84277138099697,145.11858445999695 -37.84279194299692,145.11855876200383 -37.842803877997696,145.11851761299704 -37.84281890099806,145.11847552300412 -37.84283188199898,145.1183896399973 -37.84285542699836,145.11831745500263 -37.84287466700168,145.11824589300383 -37.84289373699695,145.1182057809981 -37.842906865997,145.11816617999605 -37.8429210220012,145.11813953800024 -37.84292878800435,145.1181216670036 -37.84293146700005,145.11810313299935 -37.842931350996075,145.11809057599675 -37.84292993400308,145.11807383699744 -37.8429266379995,145.1180518749956 -37.84292025799982,145.11802432099702 -37.84290940899551,145.11798902800314 -37.84289224500096,145.1179541509947 -37.84287493399545,145.11792551700006 -37.84286143599876,145.11789650600537 -37.842848878003096,145.1178767439989 -37.842842036997425,145.1178625049969 -37.842839448997374,145.1178476499949 -37.842839527001956,145.1178375049954 -37.84284076099703,145.11782384999714 -37.84284352599955,145.11780563400083 -37.842848460997544,145.1177819990048 -37.84285559999632,145.1177484710021 -37.84286582400319,145.1177160180025 -37.84287714699714,145.11769488499633 -37.842885748000924,145.11766653699576 -37.84289947099802,145.11763985299456 -37.84291502199728,145.1176223840045 -37.84292653999668,145.11760485599814 -37.84293967800246,145.11758961500203 -37.842953953000254,145.11758096999776 -37.842964295004,145.117571663 -37.84297933199625,145.1175688609989 -37.84298628899937,145.11756721100156 -37.84299336400363,145.11756659300394 -37.84300780000094,145.11757032399677 -37.84303722999985,145.11757135300388 -37.84305667600057,145.11756963299968 -37.84306752600361,145.11756147100573 -37.84309851399774,145.1175571820036 -37.84310839400133,145.11754866599674 -37.84312036099847,145.1175418860025 -37.843126336001056,145.11751894099916 -37.843141677001235,145.11744876300193 -37.84318265800239,145.1173916729957 -37.843219901996015,145.1173345509992 -37.84325716300384,145.11728796300227 -37.84328686900428,145.11724063399842 -37.843316292996036,145.1172208539992 -37.84332800799567,145.11719987400176 -37.843338586001096,145.1171852029955 -37.84334430800186,145.1171647800025 -37.84334945099588,145.11715269099793 -37.84335079000281,145.11713406699735 -37.84335098799641,145.11711548800497 -37.84335010500287,145.11707831799853 -37.8433482129969,145.1170349480023 -37.843347600002076,145.11699152900027 -37.843348846996946,145.11698135800137 -37.84335009500013,145.11697151899457 -37.84335221800351,145.11695255599741 -37.84335852000086,145.11691612400506 -37.843375529003985,145.1168912889957 -37.84338776599735,145.1168675950039 -37.84340064400402,145.1168528449946 -37.84341012299586,145.1168349509959 -37.8434245080017,145.1168033809966 -37.843456490998655,145.11677354999517 -37.843489385002,145.11675297199565 -37.843512747999426,145.11673369799604 -37.84353687100175,145.11672629800492 -37.84354981999935,145.11672214000274 -37.84356390199744,145.11672111399506 -37.84357357400061,145.11672188800085 -37.843586419004126,145.11672659600046 -37.843602662001516,145.11673963899705 -37.843627877999104,145.11675539099866 -37.84365243599561,145.1167701139984 -37.84367142500254,145.11678548399934 -37.84369012699726,145.11679495900236 -37.843703544000135,145.11680287499826 -37.8437186670009,145.11680785300194 -37.84373422000281,145.11680941000293 -37.84374460899924,145.11680916300168 -37.84375824000348,145.11680456100453 -37.843780073999056,145.11679595500115 -37.843801816997136,145.11678793300064 -37.84381581999895,145.11537531599586 -37.84366310699748,145.11529089099903 -37.84365336800424,145.11520374599652 -37.84364248700251,145.1149283149988 -37.84361174999622,145.11475000800493 -37.843591271996196,145.114566438997 -37.84357017600423,145.11438354299858 -37.84354915299874,145.11420024599533 -37.84352799200067,145.11402517699517 -37.843507871997886,145.11384664500335 -37.84348734499961,145.11366882399645 -37.84346597800307,145.11349045400289 -37.84344641799782,145.11331241700037 -37.843424942999626,145.11313403800008 -37.84340542500002,145.11295583999726 -37.84338492999586,145.11277770999445 -37.84336443099619,145.11260648999985 -37.84334381099567,145.11243495799752 -37.843325016997085,145.11210979000572 -37.84328762699889,145.1120683100015 -37.84328285799862,145.1118953420016 -37.84326200499656,145.11176657700454 -37.843248120998496,145.11161596600033 -37.84322984700019,145.11144446299713 -37.84321098900294,145.11127330099836 -37.84319033499808,145.1111018290032 -37.84317148999734,145.11093066500524 -37.84315085800345,145.1107420730024 -37.84313004600358,145.1101764090017 -37.84306990099927))', current_timestamp());

num_affected_rows,num_inserted_rows
10,10


In [0]:
%sql
CREATE OR REPLACE TABLE dw_vicroads_demo.default.sample_points_geom AS
SELECT * except (location), st_setsrid(st_geomfromtext(location), 4326) as geometry FROM dw_vicroads_demo.default.sample_points 

num_affected_rows,num_inserted_rows


In [0]:
%sql
DESCRIBE EXTENDED dw_vicroads_demo.default.sample_points_geom

col_name,data_type,comment
id,bigint,null
name,string,null
created_date,timestamp,null
geometry,geometry(4326),null
,,
# Delta Statistics Columns,,
Column Names,"id, name, created_date, geometry",
Column Selection Method,first-32,
,,
# Detailed Table Information,,
